# Problem 3 - Solving a Rate Law Problem with Object Oriented Programming

Design and implement a class, `Experiment`, to read in and store a simple series of $\left(x,y\right)$ data as `pylab` (i.e., NumPy) arrays from a text file. Include in your class methods for transforming the data series by some simple function (e.g., $x'=\ln{x},y'=1/y$) and to perform a linear leastsquares regression on the transformed data (returning the gradient and intercept of the best-fit line, $y'_{\text{fit}}$). NumPy provides methods for performing linear regression, but for this exercise the following equations can be implemented directly:

$$
\begin{aligned}
    m &= \frac{\overline{xy}-\overline{x}\overline{y}}{\overline{x^2}-\overline{x}^2}\\
    c &= \overline{y}-m\overline{x}
\end{aligned}
$$

where the bar notation, $\overline{\cdot}$, denotes the arithmetic mean of the quantity under it. (*Hint:* use `pylab.mean(arr)` to return the mean of array `arr`.)

Chloroacetic acid is an important compound in the synthetic production of pharmaceuticals, pesticides and fuels. At high concentration under strong alkaline conditions its hydrolysis may be considered as the following reaction:

$$
\text{ClCH}_2\text{COO}^- + \text{OH}^- \leftrightarrow \text{HOCH}_2\text{COO}^- + \text{Cl}^-
$$

Data giving the concentration of $\text{ClCH}_2\text{COO}^-$, $c$ (in M), as a function of time, $t$ (in s), are provided for this reaction carried out in excess alkalai at five different temperatures in data files `caa-T.text` (`T = 40, 50, 60, 70, 80` in $^\circ\text{C}$): these may be found from https://scipython.com/static/media/problems/P4.6/caa-T.zip. The reaction is known to be second order and so obeys the integrated rate law.

$$
\frac{1}{c} = \frac{1}{c_0} + kt
$$

where $k$ is the effective rate constant and $c_0$ the initial ($t=0$) concentration of chloroacetic acid.

Use your `Experiment` class to interpret these data by linear regression of $1/c$ against $t$, determining $m(\equiv k)$ for each temperature. Then, for each value of $k$, determine the activation energy of the reaction through a second linear regression of $\ln{k}$ against $1/T$ in accordance with the Arrhenius law:

$$
k = Ae^{-E_a/RT} \Rightarrow \ln{k} = \ln{A}-\frac{E_a}{RT}
$$

where $R=8.314~\text{J/mol-K}$ is the gas constant. Note: the temperature must be in Kelvin.

# 1. Learning Objectives
This problem will be used to introduce myself to object oriented programming, which I think will be pretty useful especially for handling and using data (e.g., rate data for obtaining rate laws, and process data for developing empirical models of processes). Might also be useful for developing programs (for ChE application) as well - I mean it is useful for developing programs for general applications but, for ChE applications, so far I've only been using *procedural* programming. After this problem, I should be introduced to:
- Concepts surrounding the topic: *object, class, attributes, methods, instantiation, inheritance*. 
- Designing and implementing classes
- Creating methods
- Storing data series as pylab arrays from a text file.

# 2. Notes

## 2.1. Object-Oriented Programming Basics 

One of the reasons why OOP is so popular is because it is helpful in dissecting problems.

### Example - The `BankAccount` Class

Suppose we own a retail bank. To manage this bank using OOP, we could create two classes: a `BankAccount` class containing details about the accounts such as number, owner, status, etc., and a `customer` class containing details about the customer such as the name, address, birthdate, etc. 

![Prob3-fig1](./images/Prob3-fig1.jpg)

A bank account (object) is an instance of the class `BankAccount` which will dictate its attributes (what it is). When registered, a customer (object) is instantiated by the class `customer` which will also dictate its attributes.

Now let us try to think about the rate law problem - what is our object, its attributes and methods, and the class. Suppose you want to determine the rate law of a reaction using experimental data (which is our main problem). Using OOP, you could define a class, `Experiment`, that has attributes such as the variables time, and concentration, and a method for manipulating these attributes, turning them into something useful in answering the problem (e.g., performing linear regression on the data).

In that case the object is the experimental data; attributes could be the time array and the corresponding concentration array; and the methods could be performing a linear regression on the data. Thus, the data contained in the `caa-40.txt` file is an instance of the `Experiment` class. Its attributes are its time and concentration vector, and it exposes the `linreg()` method to spit out the gradient and intercept.

### Inheritance

An important concept in OOP is *inheritance*. To illustrate this, in the previous example there are multiple types of bank accounts, e.g., savings account and checking account. Now, you could make a class for each of them (a `SavingsAccount` class and a `CheckingAccount` class); the two classes would have some common attributes (e.g., account number, and customer) and methods (e.g., deposit, and withdraw). However, one way to approach this, which avoids repeating the common attributes and methods, is to define them as two *subclasses* of the base class `BankAccount`.

The base class `BankAccount` would contain the attributes and methods common in all bank accounts; these attributes and methods of the base class would then be *inherted* by the specialized subclasses. Meanwhile, the `SavingsAccount` subclass could have an `interest_rate` attribute since they pay interest on money in savings, but the `CheckingAccount` subclass could have a `annual_fees` attribute, characteristic of a checking account.

![Prob3-fig2](./images/Prob3-fig2.jpg)

Note that you can just build a base class, but only leave it in the background; so, you might just want to build it as a template for the subclasses but not actually add/use any instance of it. In that case, the base class is called an *abstract class*.

## 2.2. Defining and using classes in Python

Classes are defined using the `class` keyword followed by the name of the class. Its content (including methods and attributes) are then indented in a block following its declaration. Some conventions are:

1. The name of the class is written in *CamelCase*
2. Similar to defining a function, a docstring is created to describe what the class is, its attributes, and methods.

A method is defined in the same way as functions - using the `def` keyword - except that the first argument should be the keyword `self`, used to refer to the object itself when it wants to use its own attributes and created methods within the body of the current method. 

### Example - Defining the Bank Account Class

Here I'll define the `BankAccount` class earlier to demonstrate (to myself haha) how it is done. The attributes I'm going to include are `customer`, `account_number`, and `balance`; while the methods are `deposit()`, `withdraw()`, and `check_balance()`.

In [36]:
# bank_account.py

class BankAccount:
    '''
    A class to be used to manage all registered bank accounts using a program.
    the currency is in dollars '$'
    '''
    
    currency = '$'
    
    def __init__(self, customer, account_number, balance=0):
        '''
        Method to initialize the attributes customer, account number, and balance
        '''
        
        self.customer = customer
        self.account_number = account_number
        self.balance = balance
        
    def deposit(self, amount):
        '''
        Method for depositing money to the bank account
        '''
        
        if amount > 0:
            self.balance += amount
        else:
            print("Invalid deposit amount. Please try again.")
        
    def withdraw(self, amount):
        '''
        Method for withdrawing money from the bank account
        '''
        
        if amount > 0:
            if self.balance > amount:
                self.balance -= amount
            else:
                print("Insufficient funds!")
        else:
            print("Invalid withdraw amount. Please try again.")
            
    def check_balance(self):
        '''
        Method for checking the balance of an account
        '''
        
        print('The balance of account number {:d} is {:s}{:f.2}'
              .format(self.account_number, self.currency, self.balance))

To use this class, we simply save it as a .py file and then import it in our future codes the same way we do for NumPy and other libraries:

In [1]:
import sys 
import os
sys.path.append(os.path.abspath("/Users/skulp/Documents/4th-Year-Thesis/PDC-Ch2/scripts/"))

from bank_account import BankAccount

## 2.3. Instantiating Objects

To create and instance of a class, we simply use the syntax `object = ClassName(args)` where the `args` should contain the attributes of the object.

For example, suppose John Doe has registered a bank account and he made an initial deposit of $100; the account number generated for him was 21457288. Let's create a `BankAccount` object to manage his bank account.

In [2]:
my_account = BankAccount('John Doe', 21457288, 100)

## 2.4. Methods and Attributes

There are two types of attributes in a class:
1. `customer`, `account_number`, and `balance` (the ones contained in the `__init__()` method) are called *Instance variables*: these are attributes that vary from object to object. So, since each bank account has a unique account number, we must create the attribute `account_number` as an instance variable.
2. the `currency` attribute is called an *Class variable*: An attribute that does not depend on the instances. Note that this is still an attribute of the instances, although constant.

Both methods and attributes can be accessed using the `Obj.attr` notation. For instance, we have the code below:

In [7]:
print(my_account.customer) #prints the customer's name
print(my_account.account_number) #prints the account number
print(my_account.balance) #prints the amount of money in the account prior to depositing

my_account.deposit(60) #John Doe deposits $60
my_account.check_balance() #rechecking the balance by printing

John Doe
21457288
100
The balance of account number 21457288 is $160.000000
